In [31]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from textblob import TextBlob
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
from scipy.ndimage.interpolation import rotate
import re
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from wordcloud import WordCloud
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

In [148]:
df=pd.read_pickle('wine_data.pkl')

In [153]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,cleaned,features
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,0.0,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,aroma includ tropic fruit broom brimston dri h...,"[dri, herb, offer, citru, dri]"
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,ripe fruit wine smooth still structur firm tan...,"[ripe, smooth, structur, firm, tannin, juici]"
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,tart snappi flavor fruit flesh rind domin gree...,"[tart, green, crisp]"
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,fruit rind fruit pith fruit blossom start arom...,"[note, give]"
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,much like regular bottl 2012 come across rathe...,"[bottl, tannic, herbal]"


In [126]:
df.isnull().sum()

country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
cleaned                      0
features                     0
dtype: int64

In [152]:
df['variety']=df.variety.fillna('none')
df['price']=df.price.fillna(0)


In [129]:
strBase = "ripe fruit wine firm tannin juici red berri"
lstTar = ["ripe",  "green"]

def isInStr(argLstTar, argStrBase):
    if any ((x in argStrBase) for x in argLstTar):
        return True
    else:
        return False
df[df["cleaned"].apply(lambda x : isInStr(lstTar, x))]


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,cleaned,features
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,aroma includ tropic fruit broom brimston dri h...,"[dri, herb, offer, citru, dri]"
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,ripe fruit wine smooth still structur firm tan...,"[ripe, smooth, structur, firm, tannin, juici]"
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,tart snappi flavor fruit flesh rind domin gree...,"[tart, green, crisp]"
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,fruit fruit aroma show typic navarran whiff gr...,"[green, herb, mouth, full, bodi, spici, herbal..."
14,US,Building on 150 years and six generations of w...,NaN,87,12.0,California,Central Coast,Central Coast,Matt Kettmann,@mattkettmann,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou,build 150 year six gener winemak tradit wineri...,"[year, style, tart, green, ripe]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129948,Argentina,Raspberry and cassis aromas are fresh and upri...,Pedernal,90,43.0,Other,San Juan,NaN,Michael Schachner,@wineschach,Finca Las Moras 2010 Pedernal Malbec (San Juan),Malbec,Finca Las Moras,fruit cassi aroma fresh upright show wiri gras...,"[fresh, full, bodi, fresh, herbal, green, tannic]"
129955,New Zealand,"Like Dog Point's 2011 Chardonnay, this wine is...",NaN,90,40.0,Marlborough,NaN,NaN,Joe Czerwinski,@JoeCz,Dog Point 2012 Chardonnay (Marlborough),Chardonnay,Dog Point,like dog point 2011 chardonnay wine domin citr...,"[hint, toast, note, citru, crisp]"
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,granit soil brand grand cru vineyard produc st...,"[vineyard, structur, rich, ripe, dri, hint, we..."
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,rich beauti dri wine also offer consider fresh...,"[rich, dri, offer, fresh, ripe, spice]"


In [130]:
lstFind = {"feature": ["ripe", "green"], "price":[30, 50]}
def find(argDicTarget, argDfData):
    return (df[  df["cleaned"].apply(lambda x : isInStr(argDicTarget["feature"], x))
                & df["price"].apply(lambda x : int(x) > argDicTarget["price"][0])
                & df["price"].apply(lambda x : int(x) < argDictTarget["price"][1])
               ])

find(lstFind, df)


ValueError: cannot convert float NaN to integer

In [181]:
wine_dict = {'features' : [],'variety' : [],'low_price' : 0, 'high_price' : 9999}


In [182]:

def Search_Wine(wine_dict, wine_df):
    features_count = {}
    i=0
    for token in wine_df['cleaned']:
        if i not in features_count.keys():
            features_count[i] = 0 
        token = token.split()
        for w in wine_dict['features']:
            if w in token:
                features_count[i]= features_count[i] + 1
        i = i + 1
    wine_df['feature_count']=features_count.values()
    #======================================================
    variety_count = {}
    i=0
    for token in wine_df['variety']:
        if i not in variety_count.keys():
            variety_count[i] = 0 
        token = token.split()
        for w in wine_dict['variety']:
            if w in token:
                variety_count[i]= variety_count[i] + 1
        i = i + 1
    wine_df['variety_count']=variety_count.values()
    #======================================================
    wine_search_df = wine_df[(wine_df.price>wine_dict['low_price'])&(wine_df.price<wine_dict['high_price'])]
    return wine_search_df

In [183]:
wine_search_df = Search_Wine(wine_dict,df)
wine_search_df.sort_values(by=['variety_count','feature_count','points','price'],ascending=False).head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,cleaned,features,feature_count,variety_count
111753,France,"Almost black in color, this stunning wine is g...",NaN,100,1500.0,Bordeaux,Pauillac,NaN,Roger Voss,@vossroger,Château Lafite Rothschild 2010 Pauillac,Bordeaux-style Red Blend,Château Lafite Rothschild,almost black color stun wine gorgeou rich dens...,"[rich, tannin]",0,0
111755,France,This is the finest Cheval Blanc for many years...,NaN,100,1500.0,Bordeaux,Saint-Émilion,NaN,Roger Voss,@vossroger,Château Cheval Blanc 2010 Saint-Émilion,Bordeaux-style Red Blend,Château Cheval Blanc,finest cheval blanc mani year quit simpli magn...,"[year, blend, structur, tannin, balanc, well]",0,0
122935,France,"Full of ripe fruit, opulent and concentrated, ...",NaN,100,848.0,Bordeaux,Pessac-Léognan,NaN,Roger Voss,@vossroger,Château Haut-Brion 2014 Pessac-Léognan,Bordeaux-style White Blend,Château Haut-Brion,full ripe fruit opul concentr fabul impress wi...,"[full, ripe, concentr, balanc, ripe, wood, age...",0,0
114972,Portugal,"A powerful and ripe wine, strongly influenced ...",Nacional Vintage,100,650.0,Port,NaN,NaN,Roger Voss,@vossroger,Quinta do Noval 2011 Nacional Vintage (Port),Port,Quinta do Noval,power ripe wine strongli influenc tannin super...,"[ripe, tannin, ripe, structur, long, rich, tex...",0,0
89729,France,This new release from a great vintage for Char...,Le Mesnil Blanc de Blancs Brut,100,617.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Salon 2006 Le Mesnil Blanc de Blancs Brut Char...,Chardonnay,Salon,new releas great vintag chardonnay champagn pe...,"[intens, crisp, miner, note, hint, complex, ag...",0,0
